In [1]:
# Objective : create ETL process for Chess.com data

In [15]:
# Imports 
import chessdotcom as chess
import pandas as pd
import numpy as np

In [16]:
# Define player profile: warning it is case sensitive
player_profile = 'aledbs' 

In [17]:
def get_player_timeline(player_profile:str):
    '''
    Get list of all years and months were the player played games on chess.com 

        Parameters:
            player_profile (str): Profile of the player
        
        Returns: 
            List of lists with Year and Month where player played
    '''
    list_archives_games = chess.get_player_game_archives(player_profile).json['archives']
    return [i.split('/')[-2:] for i in list_archives_games]

In [18]:
def get_all_player_games(player_profile:str, player_timeline:list):
  '''
    Get list of all years and months were the player played games on chess.com 

        Parameters:
            player_profile: Profile of the player
            player_timeline: List of lists with years and months played
        
        Returns: 
            game_df (DataFrame): Df containing all games played by the player since day 1
    '''
  game_df = pd.DataFrame()

  for month_year in player_timeline:
      monthly_game = chess.get_player_games_by_month(player_profile, month_year[0], month_year[1])
      monthly_game_df = pd.DataFrame(monthly_game.json['games'])
      game_df = pd.concat([game_df,monthly_game_df])
      
  return game_df

In [ ]:
def convert_pgn_to_list(pgn_str:str):
  '''
  Convert a pgn string into a list of lists containing key-value of a pgn string 

      Parameters:
          pgn_str: A pgn string 
      
      Returns: 
          split_pgn_by_column_value (list): List containing key-value of a pgn string 
  '''
  split_pgn_in_lines = str(pgn_str).replace('[',"").replace(']',"").replace('"',"").split('\n')
  split_pgn_by_column_value = [item.split(' ', maxsplit=1) for item in split_pgn_in_lines[:-2]]
  split_pgn_by_column_value.append(['Moves', split_pgn_in_lines[-2]])

  return split_pgn_by_column_value

def convert_pgn_to_columns(df:pd.DataFrame):
  '''
    Extract pgn information into new columns of existing DataFrame  

        Parameters:
            df: DataFrame with games data including a column name 'pgn' 
        
        Returns: 
            concat_pgn_df (DataFrame): Input Dataframe with new columns extracted from pgn 
    '''
  
  game_df = df
  game_df = game_df[game_df['rules'] != 'bughouse']

  game_df['pgn'] = game_df['pgn'].apply(convert_pgn_to_list)

  concat_pgn_df = pd.DataFrame()

  for item in game_df['pgn']:
    columns = [subitem[0] for subitem in item]
    values = [subitem[-1] for subitem in item]
    item_concat_pgn_df = pd.DataFrame(values).transpose()
    item_concat_pgn_df.columns = columns
    concat_pgn_df = pd.concat([concat_pgn_df, item_concat_pgn_df])
  
  return concat_pgn_df


In [ ]:
def data_transformation(game_df, player_profile):
    df = game_df.copy()

    df.reset_index(inplace = True, drop = True)

    df['Date'] = pd.to_datetime(df['Date'])
    df['UTCDate'] = pd.to_datetime(df['UTCDate'])
    df['EndDate'] = pd.to_datetime(df['EndDate'])
    df['EndTime'] = pd.to_datetime(df['EndTime']) 
    df['StartTime'] = pd.to_datetime(df['StartTime'])
    df['WhiteElo'] = df['WhiteElo'].astype(int)
    df['BlackElo'] = df['BlackElo'].astype(int)

    df['TerminationStatus'] = pd.Series(dtype=str)
    df['TerminationStatus'][df['Termination'].str.contains(player_profile)] = 'Win'
    df['TerminationStatus'][df['Termination'].str.contains('won') & ~df['Termination'].str.contains(player_profile, na=False)] = 'Lose'
    df['TerminationStatus'][df['Termination'].str.contains('drawn')] = 'Draw'
    
    df['PlayerSide'] = pd.Series(dtype=str)
    df['PlayerSide'][df['White']==player_profile] = 'White'
    df['PlayerSide'][df['White']!=player_profile] = 'Black'

    df['PlayerElo'] = pd.Series(dtype=int)
    df['PlayerElo'][df['White']==player_profile] = df['WhiteElo'][df['White']==player_profile]
    df['PlayerElo'][df['White']!=player_profile] = df['BlackElo'][df['White']!=player_profile]

    df['OpponentElo'] = pd.Series(dtype=int)
    df['OpponentElo'][df['White']==player_profile] = df['BlackElo'][df['White']==player_profile]
    df['OpponentElo'][df['White']!=player_profile] = df['WhiteElo'][df['White']!=player_profile]

    df['eloDiff'] = pd.Series(dtype=float)
    df['eloDiff'] = df['PlayerElo'] - df['OpponentElo']
    
    df['Duration'] = df['EndTime'] - df['StartTime']

    df['TerminationType'] = df['Termination'].apply(lambda x: str(x).split('drawn')[-1] if str(x).split(' won ')[-1] is None else str(x).split(' won ')[-1])
    df['Opening'] = df['ECOUrl'].str.replace('https://www.chess.com/openings/','')


    return df


In [ ]:
player_timeline = get_player_timeline(player_profile)

In [ ]:
game_df = get_all_player_games(player_profile, player_timeline)

In [ ]:
game_df = convert_pgn_to_columns(game_df)

In [266]:
game_df = data_transformation(game_df, player_profile)

C:\Users\User\AppData\Local\Temp\ipykernel_8208\3160698797.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TerminationStatus'][df['Termination'].str.contains(player_profile)] = 'Win'
C:\Users\User\AppData\Local\Temp\ipykernel_8208\3160698797.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TerminationStatus'][df['Termination'].str.contains('won') & ~df['Termination'].str.contains(player_profile, na=False)] = 'Lose'
C:\Users\User\AppData\Local\Temp\ipykernel_8208\3160698797.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documenta

In [270]:
move = game_df['Moves'].iloc[0]
print(move)

1. e4 {%clk 0:09:59.4} 1... e5 {%clk 0:09:55.5} 2. Nf3 {%clk 0:09:58} 2... Nc6 {%clk 0:09:50.6} 3. Nc3 {%clk 0:09:54.9} 3... Nf6 {%clk 0:09:47.6} 4. Bc4 {%clk 0:09:49.5} 4... h6 {%clk 0:09:44} 5. d3 {%clk 0:09:47} 5... d6 {%clk 0:09:41.4} 6. O-O {%clk 0:09:45.6} 6... Be7 {%clk 0:09:38.1} 7. Be3 {%clk 0:09:07.8} 7... Bg4 {%clk 0:09:34.1} 8. Qd2 {%clk 0:08:53.9} 8... Bxf3 {%clk 0:09:31.3} 9. gxf3 {%clk 0:08:52.2} 9... O-O {%clk 0:09:28.7} 10. Rae1 {%clk 0:08:35.1} 10... Qd7 {%clk 0:09:24.6} 11. Kh1 {%clk 0:08:05.9} 11... Qh3 {%clk 0:09:15} 12. Rg1 {%clk 0:08:04.6} 12... Qxf3+ {%clk 0:08:44.7} 13. Rg2 {%clk 0:07:47.3} 13... Ng4 {%clk 0:08:34} 14. Qe2 {%clk 0:06:54.7} 14... Qxe2 {%clk 0:05:17.6} 15. Rxe2 {%clk 0:06:50.5} 15... Nd4 {%clk 0:04:58.4} 16. Bxd4 {%clk 0:06:34.4} 16... exd4 {%clk 0:04:55.1} 17. Rxg4 {%clk 0:06:25.8} 17... dxc3 {%clk 0:04:52.3} 18. bxc3 {%clk 0:06:24.2} 18... Rfe8 {%clk 0:04:44.1} 19. f3 {%clk 0:06:17.4} 19... c6 {%clk 0:04:32.7} 20. Reg2 {%clk 0:06:15.6} 20... g6

In [353]:
import re

pattern_clk = r'{%clk\s(.*?)\}'
clk = re.findall(pattern_clk, move)
print(clk)

pattern_moves = r'\d+\.\s*(.*?)\s*{'
moves = re.findall(pattern_moves, move)
for index, res in enumerate(moves):
    moves[index] = res.split(' ')[-1]

print(moves)

white_moves = [i for index, i in enumerate(moves) if index % 2 == 0]
white_clk = [i for index, i in enumerate(clk) if index % 2 == 0]

black_moves = [i for index, i in enumerate(moves) if index % 2 == 1]
black_clk = [i for index, i in enumerate(clk) if index % 2 == 1]

game_moves = pd.DataFrame({'w_moves': white_moves, 'w_clk': white_clk, 'b_move': black_moves, 'b_clk': black_clk})
game_moves

['0:09:59.4', '0:09:55.5', '0:09:58', '0:09:50.6', '0:09:54.9', '0:09:47.6', '0:09:49.5', '0:09:44', '0:09:47', '0:09:41.4', '0:09:45.6', '0:09:38.1', '0:09:07.8', '0:09:34.1', '0:08:53.9', '0:09:31.3', '0:08:52.2', '0:09:28.7', '0:08:35.1', '0:09:24.6', '0:08:05.9', '0:09:15', '0:08:04.6', '0:08:44.7', '0:07:47.3', '0:08:34', '0:06:54.7', '0:05:17.6', '0:06:50.5', '0:04:58.4', '0:06:34.4', '0:04:55.1', '0:06:25.8', '0:04:52.3', '0:06:24.2', '0:04:44.1', '0:06:17.4', '0:04:32.7', '0:06:15.6', '0:04:27.9', '0:05:19.8', '0:04:18.2', '0:05:16.1', '0:04:14.8', '0:05:15', '0:04:08.1', '0:04:32.7', '0:02:18.1', '0:04:27.9', '0:02:12.8', '0:04:22.6', '0:02:03.6', '0:04:20.3', '0:02:01.3', '0:04:18.9', '0:01:54.4', '0:04:12.9', '0:01:48.3', '0:04:07.6', '0:01:44.7', '0:03:40.4', '0:01:36.6', '0:03:37.7', '0:01:21.5', '0:03:34.1', '0:01:18.8', '0:03:32.4', '0:01:15', '0:03:27.5', '0:01:04.9', '0:03:04', '0:01:03', '0:03:02.1', '0:00:59.2', '0:03:00.7', '0:00:57.8', '0:02:47.7', '0:00:55.2', '0:

,w_moves,w_clk,b_move,b_clk
0,e4,0:09:59.4,e5,0:09:55.5
1,Nf3,0:09:58,Nc6,0:09:50.6
2,Nc3,0:09:54.9,Nf6,0:09:47.6
3,Bc4,0:09:49.5,h6,0:09:44
4,d3,0:09:47,d6,0:09:41.4
5,O-O,0:09:45.6,Be7,0:09:38.1
6,Be3,0:09:07.8,Bg4,0:09:34.1
7,Qd2,0:08:53.9,Bxf3,0:09:31.3
8,gxf3,0:08:52.2,O-O,0:09:28.7
9,Rae1,0:08:35.1,Qd7,0:09:24.6
